In [2]:
!pip install gkeepapi

  Using cached gkeepapi-0.14.2-py2.py3-none-any.whl (26 kB)
  Using cached gpsoauth-1.0.2-py3-none-any.whl (6.6 kB)
  Obtaining dependency information for pycryptodomex>=3.0 from https://files.pythonhosted.org/packages/8a/db/12a08d50aa8586c2e32b2b4461b89c0de057d1c637a13ac8b785dfd55327/pycryptodomex-3.18.0-cp35-abi3-win_amd64.whl.metadata
Using cached pycryptodomex-3.18.0-cp35-abi3-win_amd64.whl (1.7 MB)


In [3]:
import gkeepapi
import keyring
import os
import json
from secret_values import *

In [4]:
keep = gkeepapi.Keep()
state = None

In [5]:
print("Loading state")
state = None
if os.path.exists("state"):
    # Load cache
    fh = open('state', 'r')
    state = json.load(fh)
    keep.restore(state)
print("State loaded")

Loading state


In [6]:
print("Logging in")
token = keyring.get_password('google-keep-token', email)
if token is None:
    success = keep.login(email, app_password, state=state if state is not None else None)

    if success:
        token = keep.getMasterToken()
        keyring.set_password('google-keep-token', email, token)
else:
    keep.resume(email, token, state=state if state is not None else None)
print("Logged in")

Logging in


In [7]:
def syncAndDump():
    global state
    print("Syncing")
    keep.sync()
    
    print("Dumping Sync")
    # Store cache
    state = keep.dump()
    fh = open('state', 'w')
    json.dump(state, fh)
    print("Dumped")

syncAndDump()

Syncing
Dumping Sync


In [8]:
from datetime import datetime, timedelta

def getDayTitle(days_into_the_future=0):
    tomorrow = datetime.now() + timedelta(days=days_into_the_future)
    
    # Convert the day and month to their corresponding string representations
    days = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
    months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    
    day_str = days[tomorrow.weekday()]
    month_str = months[tomorrow.month - 1]
    
    tommorow_title = f"{day_str} {tomorrow.day} {month_str}"  
    return tommorow_title

In [12]:
# https://gkeepapi.readthedocs.io/en/latest/#caching-notes
def clone(from_title, to_title):
    print(f"Cloning from: {from_title}")

    # Find by labels
    gnotes = keep.find(labels=[keep.findLabel('daily')], archived=False, trashed=False)
    
    today_note = None
    tommorow_note = None
    
    tommorow_found = False
    today_found = False
    
    for gnote in gnotes:
    
        if gnote.title.lower().find(from_title.lower()) > -1:
            print("found today's note")
            today_note = gnote
            today_found = True
    
        if gnote.title.lower().find(to_title.lower()) > -1:
            print("tommorow's note already exists")
            tommorow_note = gnote
            tommorow_found = True
    
        if today_found and tommorow_found:
            break

    if not today_found:
        print("Oops, couldn't find the note to clone")
    else:

        # pre-processing: if tommorow note doesn't exist, create it
        if not tommorow_found:
            tommorow_note = keep.createList(to_title, [])
            tommorow_note.pinned = True
            tommorow_note.color = gkeepapi.node.ColorValue.Green
            label = keep.findLabel('daily')
            tommorow_note.labels.add(label)

        
        for today_entry in today_note.items:

            # side case: if we just created tommorow's note then let's just keep filling up
            exists_already = False
            if tommorow_found:
                # step 1: check if it already exists in new note, if so then don't even bother
                for tommorow_entry in tommorow_note.items:
                    if tommorow_entry.text == today_entry.text:
                        
                        # side case: if they're both indented & equal, let's make sure they r from the same parent cause we r allowing same notes if they belong to different parents
                        if tommorow_entry.indented and today_entry.indented:
                            if tommorow_entry.parent_item.text != today_entry.parent_item.text:
                                continue
                        
                        exists_already = True
                        break

            # step 2: it doesn't exist, if same parent is in new note then append indented notes to it, otherwise just keep filling up the note
            if not exists_already:
                
                        
                if not today_entry.checked:

                    if not today_entry.indented: # it's a parent
                        b = tommorow_note.add(today_entry.text, today_entry.checked, gkeepapi.node.NewListItemPlacementValue.Bottom)
                        
                    else: # it's a child: if say today had a parent called "BUSINESS" and tommorow also has one already, then let's just append its children to that one
                        
                        tommorow_shared_parent_entry_index = -1
                        appended_to_existing_parent = False
                        for another_tommorow_entry in tommorow_note.items:
                            tommorow_shared_parent_entry_index += 1
                            
                            if not another_tommorow_entry.indented: # we only care about finding the parents
                                if another_tommorow_entry.text.lower() == today_entry.parent_item.text.lower():
                                    b = tommorow_note.add(today_entry.text, checked=today_entry.checked, sort=tommorow_shared_parent_entry_index)
                                    another_tommorow_entry.indent(b)
                                    appended_to_existing_parent = True
                                    break
                        
                            
    
    syncAndDump()

## Clone Today into Tommorow

In [13]:

from_title = getDayTitle(days_into_the_future=-1) # 0 means today
to_title = getDayTitle(days_into_the_future=0)
to_title = "SAMPLE DAY"

clone(from_title, to_title)

Cloning from: THU 10 AUG
found today's note
tommorow's note already exists
Syncing
Dumping Sync


## Clone Sample into Tommorow

In [14]:
from_title = "SAMPLE DAY"
to_title = getDayTitle(days_into_the_future=0)

clone(from_title, to_title)

Cloning from: SAMPLE DAY
found today's note
Syncing
Dumping Sync


## Sync & Dump

In [125]:
syncAndDump()

Syncing
Dumping Sync


## TO-DO
- make a function that copies all unfinished notes from yesterday to today
- make a function that puts unfinished tasks together all the notes from previous days together into a new general to do to note (only copy notes with emojis inside)